In [7]:
import json
import openai
import os
from itertools import product

In [8]:
#Set OpenAI API key

openai.api_key = 'your-api-key'

In [9]:
# Define profile attributes
age_groups = ["18-22 years old", "23-26 years old", "27-35 years old", "Prefer not to say"]
genders = ["Man", "Woman", "Non-binary genderqueer or gender non-conforming", "Prefer not to say"]
hispanic_origins = ["Yes", "No"]
ethnicities = ["American Indian or Alaska Native", "Asian", "Black or African American", "Native Hawaiian or Other Pacific Islander", "White"]
educations = ["Bachelor’s degree", "Master’s degree", "Professional degree"]
experiences = ["Less than 1 year", "1-2 years", "3-5 years", "6-10 years", "More than 10 years"]

In [10]:
# Generate all possible profiles including domain expertise
profiles = list(product(age_groups, genders, hispanic_origins, ethnicities, educations, experiences))

In [11]:
# Define the questionnaire
questionnaire = [
    {
        "question": "What is your preferred development environment?",
        "answers": {
            "option_1": "Windows",
            "option_2": "macOS",
            "option_3": "Linux",
            "option_4": "Other:"
        }
    },
    {
        "question": "How do you learn to code? Please select all that apply.",
        "answers": {
            "option_1": "Online Courses or Certification",
            "option_2": "Books",
            "option_3": "School (i.e., University, College, etc.)",
            "option_4": "Coding Bootcamp",
            "option_5": "Other:"
        }
    },
    {
        "question": "What is the biggest challenge you face as a developer?",
        "answers": {
            "option_1": "Keeping up with new technologies",
            "option_2": "Work-life balance",
            "option_3": "Understanding existing codebases",
            "option_4": "Time management"
        }
    },
    {
        "question": "When choosing a programming language for a new project you prioritize:",
        "answers": {
            "option_1": "The language's performance and scalability",
            "option_2": "The development team's familiarity with the language",
            "option_3": "The language's community support and ecosystem",
            "option_4": "The specific requirements of the project"
        }
    },
    {
        "question": "How do you communicate effectively with teammates to collaborate while adhering to the timelines?",
        "answers": {
            "option_1": "Use project management tools to assign tasks and track progress ensuring everyone is aware of deadlines.",
            "option_2": "Schedule regular meetings for updates and coordination but keep them concise to avoid taking too much time away from work.",
            "option_3": "Rely on informal chats and emails for quick updates trusting team members to manage their time efficiently.",
            "option_4": "Implement a combination of written documentation for clarity and regular check-ins for personal engagement and immediate feedback."
        }
    },
    {
        "question": "How do you ensure that you stay up-to-date with industry changes as a software developer? Please select all that apply.",
        "answers": {
            "option_1": "Regularly read industry blogs, websites, and journals to learn about the latest trends and technologies.",
            "option_2": "Attend workshops, conferences, and webinars to gain insights from industry leaders and peers.",
            "option_3": "Participate in online courses and certification programs to acquire new skills and knowledge.",
            "option_4": "Engage in open-source projects and coding communities to get hands-on experience with cutting-edge technologies."
        }
    },
    {
        "question": "How do you balance between innovation and meeting project deadlines?",
        "answers": {
            "option_1": "Prioritize innovation over deadlines",
            "option_2": "Focus solely on meeting deadlines",
            "option_3": "Allocate specific time for research and innovation",
            "option_4": "Adjust project scopes to include innovation"
        }
    },
    {
        "question": "Software development contributes to societal challenges by:. Please select all that apply",
        "answers": {
            "option_1": "Improving healthcare through technology, such as telemedicine platforms and health tracking apps",
            "option_2": "Enhancing education with e-learning platforms, interactive learning tools, and virtual classrooms",
            "option_3": "Addressing climate change with environmental monitoring systems, carbon footprint calculators, and energy efficiency applications",
            "option_4": "Supporting economic development through fintech solutions, e-commerce platforms, and digital marketplaces"
        }
    },
    {
        "question": "A company uses an AI system to monitor its employees' productivity. The AI suggests firing an employee based on low productivity, but the employee is going through a tough personal time. How should the company proceed?",
        "answers": {
            "option_1": "Follow the AI’s suggestion and fire the employee",
            "option_2": "Ignore the AI’s suggestion and keep the employee",
            "option_3": "Use the AI’s suggestion as one of several factors in the decision",
        }
    },
    {
        "question": "You're assigned to a project that requires expertise in a programming language you're not familiar with. The deadline is tight. What is your strategy to tackle this situation?",
        "answers": {
            "option_1": "Spend time upfront to learn the basics of the language",
            "option_2": "Use your existing knowledge and adapt as you go",
            "option_3": "Seek assistance from a colleague who knows the language",
            "option_4": "Suggest reallocating the project to someone with the necessary expertise",
        }
    },
    {
        "question": "You discover a critical bug right before a software release. What immediate action do you take?",
        "answers": {
            "option_1": "Proceed with the release as planned and fix the bug in a subsequent patch",
            "option_2": "Delay the release to assess and fix the bug, ensuring the product meets quality standards",
            "option_3": "Release a limited beta version to a select group of users for further testing while addressing the bug",
            "option_4": "Communicate with stakeholders about the issue and seek their input on how to proceed",
        }
    },
    {
        "question": " In a professional setting, if your team is supposed to deliver a SaaS product within a two-day timeframe, and you discover a critical bug in the software, how would you address and handle this situation?",
        "answers": {
            "option_1": "Ignore the bug and proceed with the delivery as planned, with the intention of fixing the bug in the next update.",
            "option_2": "Inform your team leader or project manager about the bug immediately, and work collaboratively to assess its impact and decide on the next steps",
            "option_3": "Delay the delivery without informing stakeholders to buy time for fixing the bug",
            "option_4": "Document the bug and send an email to the client explaining the issue, suggesting a new timeline for delivery",
            "option_5": "Prioritize the bug based on its severity and impact, and if it's critical, reallocate resources to fix it immediately, even if this means pushing back the delivery date",
        }
    },

]


In [12]:
def call_gpt(prompt, temperature=0.0):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # or whichever model you're using
            messages=[
                {"role": "system", "content": prompt}
            ],
            temperature=temperature
        )
        return response.choices[0].message['content']
    except Exception as e:
        print(f"Error in generating response: {e}")
        return None

In [14]:
# Generate responses for each profile
all_responses = []
prompt_id = 1

for profile in profiles:
    age, gender, hispanic_origins, ethnicity, education, experience = profile
    hispanic_origin_desc = "of Hispanic origin" if hispanic_origins == "Yes" else "not of Hispanic origin"
    domain_expertise = "Domain Expert" if experience in ["6-10 years", "More than 10 years"] else "Non-Domain Expert"
    profile_desc = f" age: {age}, gender: {gender}, {hispanic_origin_desc}, {ethnicity}, with a {education} and {experience} of coding experience"

    for q in questionnaire:
        # Construct the prompt for each question with its choices
        prompt = f"Imagine you are of {profile_desc} ({domain_expertise}). Answer the following question: {q['question']}?\n"
        for key, option in q['answers'].items():
            prompt += f" {option}\n"

        # Call the GPT model for each question
        response = call_gpt(prompt)

        # Determine the option_id(s) and handle 'Other' option
        option_ids = []
        other_response = None
        for key, value in q["answers"].items():
            if value in response:
                option_ids.append(key)
                if value == "Other":
                    # Extract custom answer after 'Other:'
                    other_response = response.split('Other:')[1].strip() if 'Other:' in response else None
        if not option_ids:
            option_ids.append("option_unknown")

        all_responses.append({
            "prompt_id": prompt_id,
            "prompt": prompt,
            "profile": profile_desc,
            "Hispanic_Origin": hispanic_origin_desc,
            "Domain_Expertise": domain_expertise,
            "Question": q['question'],
            "Answer": response,
            "option_ids": option_ids,  # List of option IDs
            "other_response": other_response  # Custom answer for 'Other' option
        })
        prompt_id += 1

Error in generating response: The model `gpt-4` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.


TypeError: argument of type 'NoneType' is not iterable

In [6]:
# Save the responses in the specified format
def save_json(data, filename) :
  with open(filename, 'w', encoding='utf-8') as f :
    json.dump(data, f, indent=4)

In [ ]:
save_json(all_responses, 'ecs_260_all_profiles_responses.json')